# Juliaから外部プログラムを実行する
© 2021 Shuhei Ohno
<br>License: https://opensource.org/licenses/MIT
<br>Repository: https://github.com/ohno/RunningExternalPrograms

[Calling C and Fortran Code](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/)によると, JuliaではCやFortranのルーチンを`ccall`で呼び出せます. しかし, 共有ライブラリとしてコンパイルしないとけないので, 標準入力と標準出力でデータをやり取りするメインプログラムを呼び出すことは難しいように思われます. 元のプログラムには手を加えないで, Julaiからプログラムを呼び出せるようにすることがこのノートのテーマです.

CやFortranに限らず, 外部プログラムを呼び出す方法は[Running External Programs](https://docs.julialang.org/en/v1/manual/running-external-programs/)に概ね書いてありますが, サンプルが少なく, Windows環境だといろいろ引っかかるポイントがあるので補足説明していきます. 環境は以下の通りです.

In [1]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-4650U CPU @ 1.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)


## Hello World!

さっそくですがHello World!していきましょう. 戻り値が不要な(標準出力で良い)場合は`run()`, 結果を受け取りたい場合は`read()`か`readchomp()`です. なお, コマンドを囲むのはバッククォートです. 

<strong>これはWindowsでは動きません</strong>

In [2]:
run(`echo hello`)

LoadError: IOError: could not spawn `echo hello`: no such file or directory (ENOENT)

In [3]:
read(`echo hello`, String)

LoadError: IOError: could not spawn `echo hello`: no such file or directory (ENOENT)

In [4]:
readchomp(`echo hello`)

LoadError: IOError: could not spawn `echo hello`: no such file or directory (ENOENT)

<strong>Windowsでのechoはこっち！</strong>

In [5]:
run(`cmd /C echo hello`)

hello


Process(`cmd /C echo hello`, ProcessExited(0))

In [6]:
read(`cmd /C echo hello`, String)

"hello\r\n"

In [7]:
readchomp(`cmd /C echo hello`)

"hello"

型はそれぞれ異なります.

In [8]:
println("1. ", typeof(`cmd /C echo hello`))
println("2. ", typeof(run(`cmd /C echo hello`)))
println("3. ", typeof(read(`cmd /C echo hello`, String)))
println("4. ", typeof(readchomp(`cmd /C echo hello`)))

1. Cmd
hello
2. Base.Process
3. String
4. SubString{String}


メモ帳`notepad.exe`などのプログラムを呼び出すこともできます.

In [9]:
run(`notepad`)

Process(`notepad`, ProcessExited(0))

In [10]:
run(`notepad.exe`)

Process(`notepad.exe`, ProcessExited(0))

In [11]:
run(`notepad.exe program1.f90`)

Process(`notepad.exe program1.f90`, ProcessExited(0))

## 標準出力のみ

Hello World!と同じですが, 一応確認しましょう. 次の`program1.f90`は4という数値を標準出力に返すだけのプログラムです. コンパイルして生成された`program1.exe`を呼び出して標準出力の動作を確認していきます. (ファイルは[リポジトリ](https://github.com/ohno/RunningExternalPrograms)にあります. `gfortran`がインストールされてパスが通っていれば`compile.bat`をクリックすると勝手にコンパイルされます.)

`program1.f90`
```fortran
program main
  write(6,*) 4
end program main
```

In [12]:
run(`program1`);

           4


In [13]:
run(`program1.exe`);

           4


In [14]:
run(`cmd /C program1.exe`)

           4


Process(`cmd /C program1.exe`, ProcessExited(0))

In [15]:
read(`program1.exe`, String)

"           4\r\n"

In [16]:
readchomp(`program1.exe`)

"           4"

## 標準入力から変数を１つ渡す

次の`program2.f90`は標準入力をread文で読み取り, 数値を2乗して標準出力に返すプログラムです. コンパイルして生成された`program2.exe`を呼び出して動作を確認していきます. (ファイルは[リポジトリ](https://github.com/ohno/RunningExternalPrograms)にあります. `gfortran`がインストールされてパスが通っていれば`compile.bat`をクリックすると勝手にコンパイルされます.)

`program2.f90`
```fortran
program main
  implicit none
  integer x
  read(5,*) x
  write(6,*) x**2
end program main
```
`input2.txt`
```
5
```

まず, コマンドラインから`<`によって標準入力を渡せますが, `<`は`'<'`のようにシングルクォーテーションで囲います.

In [17]:
run(`cmd /C program2.exe '<' input2.txt`)

          25


Process(`cmd /C program2.exe '<' input2.txt`, ProcessExited(0))

コマンドプロンプトを経由せずに実行するには`pipeline()`を使います. 引数`stdin`にファイル名やコマンドを与えることができます. ファイル名の時はダブルクォーテーション, コマンドはバッククォートなので気を付けてください.

In [18]:
run(pipeline(`program2.exe`, stdin="input2.txt"))

          25


Process(`program2.exe`, ProcessExited(0))

In [19]:
run(pipeline(`program2.exe`, stdin=`cmd /C echo 6`))

          36


Base.ProcessChain(Base.Process[Process(`cmd /C echo 6`, ProcessExited(0)), Process(`program2.exe`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

以下のように`open`文で標準入力を渡すこともできます. 恐らく, 結果を受け取れないようなので`run`でよいと思います. 強いて言うなら, `open`と`end`の間で`for`文を回したり, `write`, `print`,`println`などが使い分けられるなどのメリットがあります.

In [20]:
open(`program2.exe`, "w", stdout) do io
   println(io, 7)
end

          49


In [21]:
io = open(`program2.exe`, "w", stdout)
println(io, 7)
close(io)

## 標準入力から変数を２つ渡す

次の`program3.f90`は標準入力をread文で読み取り, 2つの数値の和を取って標準出力に返すプログラムです. 基本的には先ほどの例と同じですが, Fortran側のプログラムが2つの数値を読み取れるようになっています. コンパイルして生成された`program3.exe`を呼び出して動作を確認していきます. (ファイルは[リポジトリ](https://github.com/ohno/RunningExternalPrograms)にあります. `gfortran`がインストールされてパスが通っていれば`compile.bat`をクリックすると勝手にコンパイルされます.)

`program3.f90`
```fortran
program main
  implicit none
  double precision x, y
  read(5,*) x, y
  write(6,*) x + y
end program main
```
`input2.txt`
```
5
6
```

In [22]:
run(pipeline(`program3.exe`, stdin="input3.txt"))

          49
   11.000000000000000     


Process(`program3.exe`, ProcessExited(0))

In [23]:
run(pipeline(`program3.exe`, stdin=`cmd /C echo 5 7`))

   12.000000000000000     


Base.ProcessChain(Base.Process[Process(`cmd /C echo 5 7`, ProcessExited(0)), Process(`program3.exe`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

In [24]:
open(`program3.exe`, "w", stdout) do io
   println(io, "5 8")
end

   13.000000000000000     


In [25]:
input = "5
9"

open(`program3.exe`, "w", stdout) do io
   println(io, input)
end

   14.000000000000000     


In [26]:
open(`program3.exe`, "w", stdout) do io
   println(io, 5)
   println(io, 10)
end

   15.000000000000000     


配列として渡したい場合には以下のように対処します.

In [27]:
input = [5,11]
run(pipeline(`program3.exe`, stdin=`cmd /C echo $input`))

   16.000000000000000     


Base.ProcessChain(Base.Process[Process(`cmd /C echo 5 11`, ProcessExited(0)), Process(`program3.exe`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

In [28]:
input = [5,12]
open(`program3.exe`, "w", stdout) do io
    for x in input
        println(io, x)
    end
end

   17.000000000000000     


## 外部プログラムを関数として扱う

先ほどの`program3.exe`を例に解説します. このプログラムを関数として扱うには, 戻り値が必要なので`read()`か`readchomp()`を利用します. これらの結果は文字列なので, さらに`parse()`を使って数値に型変換します.

In [29]:
f(X) = parse(Float64,readchomp(pipeline(`program3.exe`, stdin=`cmd /C echo $X`)))

f (generic function with 1 method)

In [30]:
y = f([5,13])

18.0

## 外部プログラムのパラメータ最適化

ここでは[Optim.jl](https://github.com/JuliaNLSolvers/Optim.jl)を利用します. 事前に[パッケージモード](https://qiita.com/skiing_LAL10/items/0c0132a34629fbc8a91f)で`add Optim`を実行してインストールし, ノート上では`using Optim`を宣言しておく必要があります. まず, [Nelder,Mead(1965)](https://doi.org/10.1093/comjnl/7.4.308)で[Rosenbrock関数](https://en.wikipedia.org/wiki/Rosenbrock_function)を最小化する例を見てみましょう.

In [31]:
# using Pkg
# Pkg.add("Optim")
using Optim

In [32]:
rosenblock_julia(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x0 = [0.0, 0.0]
opt = optimize(rosenblock_julia, x0, NelderMead())

println(Optim.minimizer(opt))
println(Optim.minimum(opt))

[0.9999634355313174, 0.9999315506115275]
3.5255270584829996e-9


次の`program4.f90`は標準入力から変数$x,y$を受け取っての値を標準出力に返す例です. このプログラムをコンパイルした`program4.exe`をJuliaの関数としての扱い, $x,y$を最適化します.

`program4.f90`
```fortran
program main
  implicit none
  double precision x, y
  read(5,*) x, y
  write(6,*) 1d0*(x*x-y)**2d0+(1d0-x)**2
end program main
```

In [33]:
rosenblock_fortran(x) = parse(Float64,readchomp(pipeline(`program4.exe`, stdin=`cmd /C echo $x`)))
x0 = [0.0, 0.0]
opt = optimize(rosenblock_fortran, x0)

println(Optim.minimizer(opt))
println(Optim.minimum(opt))

[0.9999634355313174, 0.9999315506115275]
3.5255270584829996e-9


Juliaだけの例と, Fortranと連携した例で全く同じ結果が得られました.